In [280]:
%matplotlib qt5
import sys
import copy
from pathlib import Path
import numpy as np
import scipy.signal
import scipy.ndimage
import sunpy.io.fits
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
from scipy.special import softmax
from scipy.signal import medfilt2d

In [157]:
base_path = Path(
    '/Volumes/HarshHDD-Data/Documents/CourseworkRepo/Polcal/20220323'
)

In [158]:
write_path = base_path / 'Processed'

In [159]:
raw_flat_filenames = ['DETECTOR_1_114025_FLAT.fits', 'DETECTOR_1_114303_FLAT.fits', 'DETECTOR_1_115045_FLAT.fits']

In [160]:
raw_dark_frame = 'DETECTOR_1_114550_DARK.fits'

In [161]:
catalog_base = Path(
    '/home/harsh/CourseworkRepo/WFAComparison'
)

In [162]:
ca_ir_catalog_file = 'catalog_8662.txt'

In [163]:
def get_dark_frame():

    data, header = sunpy.io.fits.read(base_path / 'Darks' / raw_dark_frame)[0]

    mean_dark = np.mean(data, 0)

    sunpy.io.fits.write(write_path / 'dark_master.fits', mean_dark, dict(), overwrite=True)

    return mean_dark

In [164]:
def get_dark_corrected_tilt_uncorrected_flat(mean_dark):

    raw_flat = None
    for raw_flat_filename in raw_flat_filenames:
        data, header = sunpy.io.fits.read(base_path / 'Flats' / raw_flat_filename)[0]

        if raw_flat is None:
            raw_flat = data
        else:
            raw_flat = np.vstack([raw_flat, data])
    all_mod = np.mean(raw_flat, 0)

    dark_corrected_flat = all_mod - mean_dark

    return dark_corrected_flat

In [165]:
def get_x_shift(dark_corrected_flat):

    plt.close('all')

    plt.clf()

    plt.cla()

    plt.figure('Click on the slit profile (0, 1024) to trace')

    plt.imshow(dark_corrected_flat, cmap='gray', origin='lower')

    point = np.array(plt.ginput(10, 600))

    a, b = np.polyfit(point[:, 0], point[:, 1], 1)

    y1 = a * np.arange(1024) + b

    y1 = ((y1.max() + y1.min()) / 2) - y1

    plt.close('all')

    plt.clf()

    plt.cla()

    return y1

In [166]:
def get_y_shift(x_corrected_flat):

    plt.close('all')

    plt.clf()

    plt.cla()

    plt.figure('Click on the line (0, 1024) profile to trace')

    plt.imshow(x_corrected_flat, cmap='gray', origin='lower')

    point = np.array(plt.ginput(10, 600))

    a, b = np.polyfit(point[:, 1], point[:, 0], 1)

    y1 = a * np.arange(1024) + b

    y1 = ((y1.max() + y1.min()) / 2) - y1

    plt.close('all')

    plt.clf()

    plt.cla()

    return y1

In [167]:
def apply_x_shift(dark_corrected_flat, y1):
    result = dark_corrected_flat.copy()

    for i in np.arange(dark_corrected_flat.shape[1]):
        scipy.ndimage.shift(
            dark_corrected_flat[0:dark_corrected_flat.shape[0], i],
            y1[i],
            result[0:dark_corrected_flat.shape[0], i],
            mode='nearest'
        )

    plt.imshow(result, cmap='gray', origin='lower')

    plt.show()

    return result

In [168]:
def apply_y_shift(dark_corrected_flat, y1):
    result = dark_corrected_flat.copy()

    for i in np.arange(dark_corrected_flat.shape[0]):
        scipy.ndimage.shift(
            dark_corrected_flat[i, :],
            y1[i],
            result[i, :],
            mode='nearest'
        )

    plt.imshow(result, cmap='gray', origin='lower')

    plt.show()

    return result


In [169]:
def adhoc_y_shift(dark_corrected_flat, y1):
    result = dark_corrected_flat.copy()

    for i in np.arange(dark_corrected_flat.shape[0]//2):
        scipy.ndimage.shift(
            dark_corrected_flat[i, :],
            y1[i],
            result[i, :],
            mode='nearest'
        )

    plt.imshow(result, cmap='gray', origin='lower')

    plt.show()

    return result

In [170]:
mean_dark = get_dark_frame()

In [171]:
dark_corrected_flat = get_dark_corrected_tilt_uncorrected_flat(mean_dark)

In [357]:
plt.imshow(dark_corrected_flat, cmap='gray', origin='lower')

In [361]:
plt.figure('Select 50 points on bright fringe')

plt.imshow(np.log(dark_corrected_flat), cmap='gray', origin='lower')

point = np.array(plt.ginput(50, 600))

In [362]:
point = point.astype(np.int64)

In [365]:
a, b, c, d, e = np.polyfit(point[:, 1], dark_corrected_flat[point[:, 1], point[:, 0]], 4)

y = a * np.arange(512)**4 + b * np.arange(512)**3 + c * np.arange(512)**2 + d * np.arange(512) + e

# plt.plot(np.arange(512), y)
# plt.plot(point[:, 1], dark_corrected_flat[point[:, 1], point[:, 0]])
y/= y.max()
plt.imshow(dark_corrected_flat / y[:, np.newaxis], cmap='gray', origin='lower')

In [366]:
better_flat = dark_corrected_flat / y[:, np.newaxis]

In [368]:
better_flat.min()

622.1294031461733

In [371]:
plt.figure('Select 50 points on bright fringe of one of the beams')

plt.imshow(better_flat, cmap='gray', origin='lower')

point = np.array(plt.ginput(50, 600))

In [373]:
point = point.astype(np.int64)

In [376]:
a, b, c, d, e = np.polyfit(point[:, 0], better_flat[point[:, 1], point[:, 0]], 4)

y = a * np.arange(512)**4 + b * np.arange(512)**3 + c * np.arange(512)**2 + d * np.arange(512) + e

# plt.plot(np.arange(512), y)
# plt.plot(point[:, 0], better_flat[point[:, 1], point[:, 0]])
y/= y.max()
plt.imshow(better_flat / y[:, np.newaxis], cmap='gray', origin='lower')

In [377]:
medfiltered_better_flat = medfilt2d(better_flat, kernel_size=3)
plt.imshow(medfiltered_better_flat, cmap='gray', origin='lower')

In [378]:
medfiltered_better_flat.min()

0.0

In [379]:
x_shifts = get_x_shift(medfiltered_better_flat)

In [380]:
x_corrected_flat = apply_x_shift(medfiltered_better_flat, x_shifts)

In [381]:
a, b = np.where(x_corrected_flat<0)

In [382]:
x_corrected_flat[a, b] = 0

In [384]:
y_shifts = get_y_shift(x_corrected_flat)

In [385]:
y_corrected_flat = apply_y_shift(x_corrected_flat, y_shifts)

In [386]:
a, b = np.where(y_corrected_flat<0)

In [387]:
y_corrected_flat[a, b] = 0

In [388]:
np.savetxt(write_path / 'x_shifts.txt', x_shifts)

In [389]:
np.savetxt(write_path / 'y_shifts.txt', y_shifts)

In [390]:
y_corrected_flat.min()

0.0

In [395]:
plt.imshow(y_corrected_flat, cmap='gray', origin='lower')

In [391]:
mu = np.mean(y_corrected_flat, axis=0)

In [444]:
pca = PCA(n_components=22)

In [445]:
nComps = 0
nCompe = 1
Xhat = np.dot(pca.fit_transform(y_corrected_flat)[:,nComps:nCompe], pca.components_[nComps:nCompe,:])
Xhat += mu
plt.imshow(Xhat, cmap='gray', origin='lower')

In [446]:
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='black')

<BarContainer object of 22 artists>

In [447]:
plt.plot(np.median(Xhat, 0) / np.median(Xhat, 0).max())

In [343]:
a, b = np.polyfit([282, 303], [8662.164, 8661.960], 1)

In [344]:
wave = a * np.arange(512) + b

In [345]:
wave

array([8664.90342857, 8664.89371429, 8664.884     , 8664.87428571,
       8664.86457143, 8664.85485714, 8664.84514286, 8664.83542857,
       8664.82571429, 8664.816     , 8664.80628571, 8664.79657143,
       8664.78685714, 8664.77714286, 8664.76742857, 8664.75771429,
       8664.748     , 8664.73828571, 8664.72857143, 8664.71885714,
       8664.70914286, 8664.69942857, 8664.68971429, 8664.68      ,
       8664.67028571, 8664.66057143, 8664.65085714, 8664.64114286,
       8664.63142857, 8664.62171429, 8664.612     , 8664.60228571,
       8664.59257143, 8664.58285714, 8664.57314286, 8664.56342857,
       8664.55371429, 8664.544     , 8664.53428571, 8664.52457143,
       8664.51485714, 8664.50514286, 8664.49542857, 8664.48571429,
       8664.476     , 8664.46628571, 8664.45657143, 8664.44685714,
       8664.43714286, 8664.42742857, 8664.41771429, 8664.408     ,
       8664.39828571, 8664.38857143, 8664.37885714, 8664.36914286,
       8664.35942857, 8664.34971429, 8664.34      , 8664.33028

In [346]:
plt.plot(wave, np.median(Xhat, 0) / np.median(Xhat, 0).max())

In [347]:
catalog = np.loadtxt('/Volumes/HarshHDD-Data/Documents/CourseworkRepo/WFAComparison/catalog_8662.txt')

In [348]:
plt.plot(catalog[:, 0], catalog[:, 1])

In [349]:
ind_list = list()
for w in wave:
    ind_list.append(np.argmin(np.abs(catalog[:, 0] - w)))

In [350]:
ind = np.array(ind_list)

In [351]:
medprof = np.median(Xhat, 0) / np.median(Xhat, 0).max()

In [352]:
plt.plot(wave, medprof/medprof[-1], label='Observed')
plt.plot(catalog[:, 0][ind], catalog[:, 1][ind]/catalog[:, 1][ind][-1], label='BASS 2000')
plt.title('Dispersion 5.17 Angstrom per mm')
plt.legend()

In [239]:
dispersion = 1/(1000 *((8662.164 - 8661.960) / (303-281))/48)

In [240]:
dispersion

5.176470588196154

In [448]:
medprof

array([0.79545622, 1.        , 0.99340996, 0.99544676, 0.98811547,
       0.98232282, 0.97507093, 0.96917724, 0.96268887, 0.95814351,
       0.95525793, 0.94586762, 0.93836289, 0.93490071, 0.93582502,
       0.94112692, 0.94023483, 0.94933932, 0.94736724, 0.95312116,
       0.95209606, 0.95211975, 0.94929837, 0.94410578, 0.94257476,
       0.94221923, 0.94461191, 0.94917796, 0.94542263, 0.94566359,
       0.93615293, 0.94030882, 0.94054778, 0.94355609, 0.93823664,
       0.93356868, 0.9310845 , 0.9295583 , 0.92849821, 0.9298332 ,
       0.93045216, 0.93325316, 0.93154717, 0.93181492, 0.93030514,
       0.92884409, 0.92754266, 0.92540933, 0.91953914, 0.91281332,
       0.91263695, 0.9171368 , 0.91728097, 0.91433008, 0.90913606,
       0.90708527, 0.90645669, 0.90119984, 0.8981811 , 0.89321625,
       0.89026912, 0.89127139, 0.89091838, 0.88830532, 0.88251661,
       0.87700451, 0.87570968, 0.87896437, 0.87671083, 0.87367837,
       0.87427727, 0.8749671 , 0.87730307, 0.87185991, 0.86979

In [449]:
plt.imshow(y_corrected_flat / medprof, cmap='gray', origin='lower')

In [450]:
np.savetxt(write_path / 'median_profile_uncorrected.txt', medprof)

In [457]:
flat_master = y_corrected_flat / medprof
flat_master += 1

In [473]:
flat_master /= flat_master.max()
sunpy.io.fits.write(write_path / 'flat_master.fits', flat_master, dict(), overwrite=True)

In [460]:
def apply_x_shift(dark_corrected_flat, y1):
    result = dark_corrected_flat.copy()

    for i in np.arange(dark_corrected_flat.shape[1]):
        scipy.ndimage.shift(
            dark_corrected_flat[0:dark_corrected_flat.shape[0], i],
            y1[i],
            result[0:dark_corrected_flat.shape[0], i],
            mode='nearest'
        )

#     plt.imshow(result, cmap='gray', origin='lower')

#     plt.show()

    return result


def apply_y_shift(dark_corrected_flat, y1):
    result = dark_corrected_flat.copy()

    for i in np.arange(dark_corrected_flat.shape[0]):
        scipy.ndimage.shift(
            dark_corrected_flat[i, :],
            y1[i],
            result[i, :],
            mode='nearest'
        )

#     plt.imshow(result, cmap='gray', origin='lower')

#     plt.show()

    return result


In [482]:
data, header = sunpy.io.fits.read(base_path / 'Flats' / raw_flat_filenames[2])[0]
corrected_flat = np.zeros_like(data)
for i in range(data.shape[0]):
    dark_corrected_mod = np.subtract(
        data[i],
        mean_dark
    )

    x_corrected_mod = apply_x_shift(dark_corrected_mod, x_shifts)

    y_corrected_mod = apply_y_shift(x_corrected_mod, y_shifts)

    flat_corrected_mod = y_corrected_mod / flat_master

    corrected_flat[i] = flat_corrected_mod

In [470]:
dark_corrected_mod = np.subtract(
        data[0],
        mean_dark
    )

In [475]:
x_corrected_mod = apply_x_shift(dark_corrected_mod, x_shifts)
plt.imshow(x_corrected_mod,cmap='gray', origin='lower')

In [476]:
y_corrected_mod = apply_y_shift(x_corrected_mod, y_shifts)
plt.imshow(y_corrected_mod, cmap='gray', origin='lower')

In [481]:
flat_corrected_mod = y_corrected_mod / flat_master
plt.imshow(flat_corrected_mod[1:], cmap='gray', origin='lower')

In [480]:
plt.plot(flat_corrected_mod[300]/flat_corrected_mod[300].max())

In [483]:
corrected_median_profile = np.median(corrected_flat, (0, 1))

In [484]:
corrected_median_profile

array([ 9350. , 11156.5, 10631. , 10821. , 10831. , 10696.5, 10610. ,
       10581.5, 10451. , 10455.5, 10482. , 10408. , 10347. , 10265.5,
       10303. , 10535. , 10492.5, 10566. , 10805. , 10812. , 10800.5,
       10633. , 10717. , 10606.5, 10431. , 10585.5, 10558. , 10632.5,
       10760.5, 10500.5, 10306. , 10438.5, 10581. , 10444. , 10325.5,
       10261.5, 10229.5, 10240.5, 10075. , 10044.5, 10300. , 10102. ,
       10214.5, 10126. , 10228.5, 10261. , 10039. , 10333. , 10205. ,
        9990.5, 10068.5, 10419.5, 10263. , 10167. , 10260. , 10220.5,
       10170. , 10150. , 10223. , 10055.5, 10144.5, 10099.5,  9994. ,
       10045.5,  9702. ,  9584.5,  9825. ,  9864. ,  9863. ,  9575. ,
        9591.5,  9750. ,  9465. ,  9655. ,  9547. ,  9450. ,  9523. ,
        9305.5,  9358. ,  9394. ,  9282. ,  9351.5,  9253. ,  9273. ,
        9297. ,  9216.5,  9079.5,  9026.5,  9095.5,  9175. ,  9031. ,
        9127.5,  9184.5,  8916. ,  8900.5,  8979. ,  8886. ,  9054.5,
        9003. ,  894

In [485]:
plt.plot(corrected_median_profile / corrected_median_profile.max())

In [486]:
np.savetxt(write_path / 'Corrected_median_profile.txt', corrected_median_profile)

In [487]:
def get_keys_from_name(filepath):
    name = filepath.name
    name_array = name.split('_')
    primary = int(name_array[4])
    sec_array = name_array[-1].split('.')
    secondary = int(sec_array[0])
    return [primary, secondary]

In [488]:
def get_calib_key_from_name(filepath):
    keys = get_keys_from_name(filepath)
    return keys[1], keys[0]


def get_observation_key_from_name(filepath):
    keys = get_keys_from_name(filepath)
    return keys[0], keys[1]


In [489]:
def get_flat_corrected_data(
    data_files,
    x_inclination_file,
    y_inclination_file,
    flat_master,
    dark_master
):
    temp_data, first_header = sunpy.io.fits.read(data_files[0])[0]
    if len(temp_data.shape) == 3:
        resultant = np.zeros(
            shape=(
                len(data_files),
                temp_data.shape[1],
                temp_data.shape[2]
            )
        )
    else:
        resultant = np.zeros(
            shape=(
                len(data_files),
                temp_data.shape[0],
                temp_data.shape[1]
            )
        )
    dark_data, dark_header = sunpy.io.fits.read(dark_master)[0]
    flat_data, flat_header = sunpy.io.fits.read(flat_master)[0]
    shift_vertical_apply = np.loadtxt(x_inclination_file)
    shift_horizontal_apply = np.loadtxt(y_inclination_file)

    for index, data_file in enumerate(data_files):
        data, header = sunpy.io.fits.read(data_file)[0]

        if len(data.shape) == 3:
            data = data[0]

        data = data - dark_data

        data[np.where(data < 0)] = 0

        medfilt_corrected = medfilt2d(data)

        x_corrected_data = apply_x_shift(medfilt_corrected, shift_vertical_apply)

        y_corrected_data = apply_y_shift(
            x_corrected_data, shift_horizontal_apply
        )


        flat_corrected_data = y_corrected_data / flat_data

        resultant[index] = flat_corrected_data

    return resultant, first_header

In [490]:
def save_calibration_data(
    calibration_folder,
    x_inclination_file,
    y_inclination_file,
    flat_master,
    dark_master
):
    everything = calibration_folder.glob('**/*')

    calibration_files = [
        x for x in everything if x.is_file() and
        x.name.endswith('.fits')
    ]

    calibration_files.sort(key=get_calib_key_from_name)

    resultant, header = get_flat_corrected_data(
        data_files=calibration_files,
        x_inclination_file=x_inclination_file,
        y_inclination_file=y_inclination_file,
        flat_master=flat_master,
        dark_master=dark_master
    )

    sunpy.io.fits.write(
        write_path / 'calib_data_{}.fits'.format(
            calibration_folder.name
        ),
        resultant,
        header,
        overwrite=True
    )


In [493]:
x_inclination_file = write_path / 'x_shifts.txt'
y_inclination_file = write_path / 'y_shifts.txt'
flat_master = write_path / 'flat_master.fits'
dark_master = write_path / 'dark_master.fits'
calibration_folder = base_path / 'Calibration' / '115206'

In [494]:
save_calibration_data(
    calibration_folder,
    x_inclination_file,
    y_inclination_file,
    flat_master,
    dark_master
)

In [507]:
def read_file(
    calib_filename,
    x1=100, x2=200, y1=100, y2=200, BEAMSEP=256
):
    data, header = sunpy.io.fits.read(calib_filename)[0]
    PTS = [[y1, x1], [y2, x2]]
    ROI = [list(map(int, i)) for i in PTS]
    ROI_TOP = np.copy(ROI)
    ROI_BOT = np.copy(ROI_TOP)
    ROI_BOT[0][1] = ROI_TOP[0][1] - BEAMSEP
    ROI_BOT[1][1] = ROI_TOP[1][1] - BEAMSEP
    INTENSITY_TOP = np.sum(
        data[:, ROI_TOP[0][1]:ROI_TOP[1][1], ROI_TOP[0][0]:ROI_TOP[1][0]],
        (1, 2)
    )
    INTENSITY_BOT = np.sum(
        data[:, ROI_BOT[0][1]:ROI_BOT[1][1], ROI_BOT[0][0]:ROI_BOT[1][0]],
        (1, 2)
    )
    return INTENSITY_TOP, INTENSITY_BOT

In [508]:
intensity_top, intensity_bot = read_file(write_path / 'calib_data_115206.fits')

In [509]:
intensity_top.shape

(96,)

In [510]:
intensity_top = intensity_top.reshape(4, 24)
intensity_bot = intensity_bot.reshape(4, 24)

In [511]:
intensity_bot

array([[1.64671652e+07, 8.05691785e+06, 6.15013836e+06, 1.15067343e+07,
        2.31914684e+07, 3.97006372e+07, 5.84360523e+07, 7.33551841e+07,
        8.15598343e+07, 7.84251540e+07, 6.52111677e+07, 4.80836402e+07,
        2.90155928e+07, 1.43698252e+07, 5.63735795e+06, 3.83649025e+06,
        8.74650940e+06, 1.88376635e+07, 3.00638927e+07, 4.12852846e+07,
        4.85825116e+07, 4.73635999e+07, 3.92682107e+07, 2.77142457e+07],
       [1.07953621e+08, 8.94757262e+07, 6.89532162e+07, 4.96851774e+07,
        3.26623825e+07, 1.90813236e+07, 9.70009520e+06, 4.92612094e+06,
        4.86297367e+06, 8.99344230e+06, 1.70548599e+07, 2.88204751e+07,
        4.26463914e+07, 5.77775132e+07, 7.47035445e+07, 9.00646428e+07,
        1.05337975e+08, 1.18842366e+08, 1.29931200e+08, 1.37551833e+08,
        1.43326704e+08, 1.42039335e+08, 1.36680388e+08, 1.24114679e+08],
       [5.23023739e+07, 6.30037348e+07, 7.68351243e+07, 8.97890287e+07,
        9.65502067e+07, 9.48917072e+07, 8.47948511e+07, 6.9795

In [524]:
def plot_top_bottom_beams(intensity_top, intensity_bot, name='Observed'):
    plt.close('all')
    fig, axs = plt.subplots(2, 2)
    axs[0][0].plot(intensity_top[0], color='blue')
    axs[0][1].plot(intensity_top[1], color='blue')
    axs[1][0].plot(intensity_top[2], color='blue')
    axs[1][1].plot(intensity_top[3], color='blue')
    axs[0][0].plot(intensity_bot[0], color='red')
    axs[0][1].plot(intensity_bot[1], color='red')
    axs[1][0].plot(intensity_bot[2], color='red')
    axs[1][1].plot(intensity_bot[3], color='red')
    fig.savefig(write_path / '{}Intensity.pdf'.format(name), format='pdf')
    plt.show()

In [515]:
plot_top_bottom_beams(intensity_top, intensity_bot)

In [516]:
def get_modulation_matrix(config, original_wavelength=6302, wavelength=8662):

    modulation_matrix_top = None
    modulation_matrix_bottom = None

    quarter_retardation = 2 * np.pi * 0.249 * original_wavelength / wavelength
    half_retardation = 2 * np.pi * 0.249 * 2 * original_wavelength / wavelength

    qwp_matrix_func = get_waveplate_matrix(quarter_retardation)
    hwp_matrix_func = get_waveplate_matrix(half_retardation)

    top_retarder = get_linear_polarizer(1)
    bottom_retarder = get_linear_polarizer(-1)

    for conf in config:
        qwp_angle = np.radians(conf[0])
        hwp_angle = np.radians(conf[1])

        qwp_matrix = qwp_matrix_func(qwp_angle)
        hwp_matrix = hwp_matrix_func(hwp_angle)

        mueller_matrix_top = np.matmul(
            np.matmul(
                top_retarder,
                hwp_matrix
            ),
            qwp_matrix
        )

        mueller_matrix_bottom = np.matmul(
            np.matmul(
                bottom_retarder,
                hwp_matrix
            ),
            qwp_matrix
        )

        if modulation_matrix_top is None:
            modulation_matrix_top = mueller_matrix_top[0]
            modulation_matrix_top = modulation_matrix_top.reshape(1, 4)
        else:
            modulation_matrix_top = np.concatenate(
                (
                    modulation_matrix_top,
                    mueller_matrix_top[0].reshape(1, 4)
                ),
                axis=0
            )

        if modulation_matrix_bottom is None:
            modulation_matrix_bottom = mueller_matrix_bottom[0]
            modulation_matrix_bottom = modulation_matrix_bottom.reshape(1, 4)
        else:
            modulation_matrix_bottom = np.concatenate(
                (
                    modulation_matrix_bottom,
                    mueller_matrix_bottom[0].reshape(1, 4)
                ),
                axis=0
            )

    return modulation_matrix_top.astype(np.float64), \
        modulation_matrix_bottom.astype(np.float64)

In [517]:
def get_input_stokes(offset=0, total=24):
    initial_stokes = get_initial_stokes()

    input_angles = get_input_angles(offset=offset, total=total)

    retardation = 2 * np.pi * 0.249

    waveplate_matrix = get_waveplate_matrix(retardation)

    vec_waveplate_matrix = np.vectorize(waveplate_matrix)

    wp_matrices = vec_waveplate_matrix(input_angles)

    stokes_res = None

    for wp_matrice in wp_matrices:
        _res = np.matmul(wp_matrice, initial_stokes)

        if stokes_res is None:
            stokes_res = _res
        else:
            stokes_res = np.concatenate((stokes_res, _res), axis=1)

    return stokes_res.astype(np.float64)


In [518]:
def get_input_angles(offset=0, total=24):
    return np.radians(np.linspace(0, 180, total + 1)[1:] + offset)


def get_initial_stokes():
    return np.array([1, 1, 0, 0]).reshape(4, 1)

In [519]:
def get_waveplate_matrix(retardation):
    def retard_waveplate_matrix(theta):
        cos_2_theta = np.cos(2 * theta)
        sin_2_theta = np.sin(2 * theta)
        cos_del = np.cos(retardation)
        sin_del = np.sin(retardation)
        q_1 = (cos_2_theta ** 2) + (sin_2_theta ** 2 * cos_del)
        q_2 = cos_2_theta * sin_2_theta * (1 - cos_del)
        q_3 = -1 * sin_2_theta * sin_del

        u_1 = q_2
        u_2 = (cos_2_theta ** 2 * cos_del) + (sin_2_theta ** 2)
        u_3 = cos_2_theta * sin_del

        v_1 = -1 * q_3
        v_2 = -1 * u_3
        v_3 = cos_del
        return np.array(
            [
                [1, 0, 0, 0],
                [0, q_1, q_2, q_3],
                [0, u_1, u_2, u_3],
                [0, v_1, v_2, v_3]
            ],
            dtype=object
        )
    return retard_waveplate_matrix


def get_linear_polarizer(sign=1):
    return np.array(
        [
            [1, sign, 0, 0],
            [sign, 1, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
        ]
    )

In [520]:
config = [[-68.10, 17.62],[-30.73, 106.02],[62.40, 71.15],[9.76, 57.16]]

In [522]:
def get_modulated_intensity(offset=0, total=24, wavelength=8662, original_wavelength=6302):
    input_stokes = get_input_stokes(offset=offset, total=total)

    modulation_matrix_top, modulation_matrix_bottom = get_modulation_matrix(
        config, wavelength=wavelength, original_wavelength=original_wavelength
    )

    intensity_top = np.matmul(
        modulation_matrix_top, input_stokes
    )

    intensity_bot = np.matmul(
        modulation_matrix_bottom, input_stokes
    )

    return intensity_top, intensity_bot

In [523]:
theory_top, theory_bot = get_modulated_intensity(offset=0, total=24, wavelength=8662, original_wavelength=6302)

In [525]:
plot_top_bottom_beams(theory_top, theory_bot, name='Theory')

In [530]:
def get_squared_dist(source, dest):
    diff = np.abs(np.abs(source) - np.abs(dest))
    least_sq_diff = np.sum(np.square(diff))
    k = len(np.where(diff < 0.4)[0])

    return least_sq_diff - ((k - 4) * 100)

In [531]:
def normalise_modulation_matrix(mod_matrix):
    mod_matrix = np.copy(mod_matrix)
    m_0 = np.zeros(mod_matrix.shape[0])

    for i in np.arange(mod_matrix.shape[0]):
        m_0[i] = mod_matrix[i][0]
        mod_matrix[i] = mod_matrix[i] / m_0[i]

    return mod_matrix, m_0

In [532]:
def get_modulation_matrix_from_beams(offset, top_beam, bottom_beam):
    input_stokes = get_input_stokes(offset)

    input_mod = np.matmul(
        input_stokes.T,
        np.linalg.inv(
            np.matmul(
                input_stokes,
                input_stokes.T
            )
        )
    )
    modulation_matrix_top = np.matmul(
        top_beam,
        input_mod
    )

    normalised_top, m_0_top = normalise_modulation_matrix(
        modulation_matrix_top
    )

    modulation_matrix_bot = np.matmul(
        bottom_beam,
        input_mod
    )

    normalised_bot, m_0_bot = normalise_modulation_matrix(
        modulation_matrix_bot
    )

    return normalised_top, m_0_top, normalised_bot, m_0_bot

In [533]:
def is_valid_mod_matrix(mod_matrix):

    for i in np.arange(mod_matrix.shape[0]):
        for j in np.arange(mod_matrix.shape[1]):
            if not -1 <= mod_matrix[i][j] <= 1:
                return False

    return True

In [536]:
def reduce_offset(
    offset_range_start, offset_range_end,
    step, top_beam, bottom_beam, wavelength=8662, original_wavelength=6302
):

    im_top, im_bot = get_modulation_matrix(
        config, wavelength=wavelength,original_wavelength=original_wavelength
    )

    im_top = im_top.astype(np.float64)

    im_bot = im_bot.astype(np.float64)

    score = np.Inf

    res_offset = None

    res_top = None

    res_bot = None

    for offset in np.arange(
        offset_range_start, offset_range_end, step
    ):
        a, b, c, d = get_modulation_matrix_from_beams(
            offset, top_beam, bottom_beam
        )

        normalised_top, _, normalised_bot, _ = a, b, c, d

        if not is_valid_mod_matrix(normalised_top) or \
                not is_valid_mod_matrix(normalised_bot):
            continue

        curr_score = get_squared_dist(normalised_top, im_top) + \
            get_squared_dist(normalised_bot, im_bot)

        if curr_score < score:
            score = curr_score
            res_offset = offset
            res_top = normalised_top
            res_bot = normalised_bot

        sys.stdout.write(
            'Offset: {} Score: {} Min Score: {} Min Offset: {}\n'.format(
                offset, curr_score, score, res_offset
            )
        )
    return score, res_offset, res_top, res_bot

In [544]:
score, res_offset, res_top, res_bot = reduce_offset(
    0, 180,
    0.1, intensity_top, intensity_bot, wavelength=8662, original_wavelength=6302
)

Offset: 58.2 Score: -1897.516057143953 Min Score: -1897.516057143953 Min Offset: 58.2
Offset: 58.300000000000004 Score: -1897.5603120442433 Min Score: -1897.5603120442433 Min Offset: 58.300000000000004
Offset: 58.400000000000006 Score: -1897.6039459531103 Min Score: -1897.6039459531103 Min Offset: 58.400000000000006
Offset: 58.5 Score: -1897.6469597342043 Min Score: -1897.6469597342043 Min Offset: 58.5
Offset: 58.6 Score: -1897.689354273943 Min Score: -1897.689354273943 Min Offset: 58.6
Offset: 58.7 Score: -1897.7311304813034 Min Score: -1897.7311304813034 Min Offset: 58.7
Offset: 58.800000000000004 Score: -1997.772289287584 Min Score: -1997.772289287584 Min Offset: 58.800000000000004
Offset: 58.900000000000006 Score: -1997.812831646138 Min Score: -1997.812831646138 Min Offset: 58.900000000000006
Offset: 59.0 Score: -1997.8527585320767 Min Score: -1997.8527585320767 Min Offset: 59.0
Offset: 59.1 Score: -1997.8920709419463 Min Score: -1997.8920709419463 Min Offset: 59.1
Offset: 59.2 Sco

In [538]:
def get_efficiency_vector(modulation_matrix):
    demod = get_demodulation_matrix(modulation_matrix)
    return 1 / np.sqrt(
        modulation_matrix.shape[0] * np.sum(
            np.square(demod),
            axis=1
        )
    )


In [541]:
def get_demodulation_matrix(mod_matrix):
    return np.matmul(
        np.linalg.inv(
            np.matmul(
                mod_matrix.T,
                mod_matrix
            )
        ),
        mod_matrix.T
    )


In [542]:
get_efficiency_vector(res_top)

array([0.98988821, 0.57274128, 0.48421484, 0.56676075])

In [543]:
get_efficiency_vector(res_bot)

array([0.96515939, 0.58967704, 0.5479834 , 0.50895089])

In [545]:
print(res_top)

[[ 1.          0.95848727 -0.28428673  0.24451052]
 [ 1.         -0.13962905  0.20938961 -0.95732157]
 [ 1.         -0.45109351 -0.69736964 -0.15133137]
 [ 1.         -0.59592229  0.70920241  0.62016489]]


In [546]:
print(res_bot)

[[ 1.         -0.91301111  0.21409603 -0.23983151]
 [ 1.         -0.06398849 -0.08550416  0.89754634]
 [ 1.          0.62786506  0.99991607  0.21140805]
 [ 1.          0.43140924 -0.57446468 -0.48948641]]
